In [1]:
import pandas as pd
import spacy
import re
from spacy import displacy 
from spacy.matcher import Matcher 
import visualise_spacy_tree
from IPython.display import Image, display

from __future__ import unicode_literals, print_function
from spacy.lang.en import English # updated

import geopy 
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter

import folium
from folium.plugins import FastMarkerCluster

import numpy as np

In [2]:
df = pd.read_csv("nbc_df.csv")
df = df.iloc[: , 1:] #get rid of redundant ids
df

,Title,Link,Date,Content
0,'We're one cigarette away': Illegal weed farms...,https://www.nbcnews.com/news/us-news/fire-guns...,Thu Dec 02 2021 01:40:21 GMT+0000 (UTC),"LAKE ELSINORE, Calif. — After a 21⁄2-mile trek..."
1,On the front lines with hotshot teams fighting...,https://www.nbcnews.com/nightly-news/video/on-...,NaN,NaN
2,Wildfire near Rocky Mountain National Park ful...,https://www.nbcnews.com/news/us-news/wildfire-...,Sun Nov 21 2021 03:09:39 GMT+0000 (UTC),A wildfire burning for four days in steep terr...
3,Thousands of giant sequoias killed in Californ...,https://www.nbcnews.com/science/science-news/t...,Fri Nov 19 2021 20:00:52 GMT+0000 (UTC),LOS ANGELES — Lightning-sparked wildfires kill...
4,Former California professor indicted on arson ...,https://www.nbcnews.com/news/us-news/former-ca...,Fri Nov 19 2021 11:57:32 GMT+0000 (UTC),A former California professor accused of setti...
...,...,...,...,...
143,"Over 900,000 acres scorched by fires in Oregon...",https://www.msnbc.com/mtp-daily/watch/over-900...,NaN,NaN
144,Power outages hamper evacuation warnings and d...,https://www.nbcnews.com/business/business-news...,Fri Sep 11 2020 17:37:16 GMT+0000 (UTC),Danielle Mays lives in a wood cabin at the end...
145,7 deaths confirmed as wildfire rages in Northe...,https://www.nbcnews.com/news/us-news/7-bodies-...,Fri Sep 11 2020 05:30:00 GMT+0000 (UTC),Seven more deaths have been confirmed where a ...
146,Wildfire north of Sacramento is now largest in...,https://www.nbcnews.com/news/us-news/wildfire-...,Fri Sep 11 2020 01:23:00 GMT+0000 (UTC),California has never before seen a fire as lar...


In [3]:
df = df.dropna().reset_index(drop=True)
df

,Title,Link,Date,Content
0,'We're one cigarette away': Illegal weed farms...,https://www.nbcnews.com/news/us-news/fire-guns...,Thu Dec 02 2021 01:40:21 GMT+0000 (UTC),"LAKE ELSINORE, Calif. — After a 21⁄2-mile trek..."
1,Wildfire near Rocky Mountain National Park ful...,https://www.nbcnews.com/news/us-news/wildfire-...,Sun Nov 21 2021 03:09:39 GMT+0000 (UTC),A wildfire burning for four days in steep terr...
2,Thousands of giant sequoias killed in Californ...,https://www.nbcnews.com/science/science-news/t...,Fri Nov 19 2021 20:00:52 GMT+0000 (UTC),LOS ANGELES — Lightning-sparked wildfires kill...
3,Former California professor indicted on arson ...,https://www.nbcnews.com/news/us-news/former-ca...,Fri Nov 19 2021 11:57:32 GMT+0000 (UTC),A former California professor accused of setti...
4,Colorado pilot fighting wildfire was making fi...,https://www.nbcnews.com/news/us-news/colorado-...,Thu Nov 18 2021 03:52:01 GMT+0000 (UTC),The pilot of a firefighting aircraft helping t...
...,...,...,...,...
108,Oregon prepares for 'mass fatality incident' a...,https://www.nbcnews.com/news/us-news/largest-w...,Sat Sep 12 2020 15:24:00 GMT+0000 (UTC),"Officials in Oregon are preparing for a ""mass ..."
109,New California law to make it easier for forme...,https://www.nbcnews.com/news/us-news/proposed-...,Fri Sep 11 2020 20:35:00 GMT+0000 (UTC),"OAKLAND, Calif.—Gov. Gavin Newsom signed a new..."
110,Power outages hamper evacuation warnings and d...,https://www.nbcnews.com/business/business-news...,Fri Sep 11 2020 17:37:16 GMT+0000 (UTC),Danielle Mays lives in a wood cabin at the end...
111,7 deaths confirmed as wildfire rages in Northe...,https://www.nbcnews.com/news/us-news/7-bodies-...,Fri Sep 11 2020 05:30:00 GMT+0000 (UTC),Seven more deaths have been confirmed where a ...


In [4]:
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text
df["Content"] = df["Content"].apply(func=clean)
df.rename(columns={"Content":"Clean_content"}, inplace=True)
df

,Title,Link,Date,Clean_content
0,'We're one cigarette away': Illegal weed farms...,https://www.nbcnews.com/news/us-news/fire-guns...,Thu Dec 02 2021 01:40:21 GMT+0000 (UTC),"LAKE ELSINORE, Calif. After a 21⁄2 mile trek t..."
1,Wildfire near Rocky Mountain National Park ful...,https://www.nbcnews.com/news/us-news/wildfire-...,Sun Nov 21 2021 03:09:39 GMT+0000 (UTC),A wildfire burning for four days in steep terr...
2,Thousands of giant sequoias killed in Californ...,https://www.nbcnews.com/science/science-news/t...,Fri Nov 19 2021 20:00:52 GMT+0000 (UTC),LOS ANGELES Lightning sparked wildfires killed...
3,Former California professor indicted on arson ...,https://www.nbcnews.com/news/us-news/former-ca...,Fri Nov 19 2021 11:57:32 GMT+0000 (UTC),A former California professor accused of setti...
4,Colorado pilot fighting wildfire was making fi...,https://www.nbcnews.com/news/us-news/colorado-...,Thu Nov 18 2021 03:52:01 GMT+0000 (UTC),The pilot of a firefighting aircraft helping t...
...,...,...,...,...
108,Oregon prepares for 'mass fatality incident' a...,https://www.nbcnews.com/news/us-news/largest-w...,Sat Sep 12 2020 15:24:00 GMT+0000 (UTC),Officials in Oregon are preparing for a mass f...
109,New California law to make it easier for forme...,https://www.nbcnews.com/news/us-news/proposed-...,Fri Sep 11 2020 20:35:00 GMT+0000 (UTC),"OAKLAND, Calif.—Gov. Gavin Newsom signed a new..."
110,Power outages hamper evacuation warnings and d...,https://www.nbcnews.com/business/business-news...,Fri Sep 11 2020 17:37:16 GMT+0000 (UTC),Danielle Mays lives in a wood cabin at the end...
111,7 deaths confirmed as wildfire rages in Northe...,https://www.nbcnews.com/news/us-news/7-bodies-...,Fri Sep 11 2020 05:30:00 GMT+0000 (UTC),Seven more deaths have been confirmed where a ...


In [5]:
#Don't init nlp in sentences function so it doesn't load a lot of times.
NLP_SENTENCES = English()
NLP_SENTENCES.add_pipe('sentencizer') 

def sentences(text):
    # split sentences and questions
    raw_text = text
    doc = NLP_SENTENCES(raw_text)
    return [sent.text.strip() for sent in doc.sents]

print(sentences(df["Clean_content"][1]))

['A wildfire burning for four days in steep terrain near Rocky Mountain National Park in Colorado has been fully contained after scorching 147 acres, officials said Saturday.', 'The Kruger Rock Fire broke out Tuesday morning after high winds knocked down a tree onto a power line, the Larimer County Sheriff Office said.', 'It became fully contained Saturday afternoon, according to the U.S. Forest Service.', 'The fire near Estes Park, a town of around 6,000 residents approximately 50 miles northwest of Denver, spread quickly amid warmer than normal temperatures and drought conditions in much of Colorado and the West.', 'The rugged terrain prevented firefighters from approaching by foot, prompting local officials to order a water drop by air.', 'An air tanker pilot died in a crash while fighting the fire.', 'The pilot, Marc Thor Olson, was attempting to drop fire suppressant over the Kruger Rock Fire when he alerted officials that conditions had become too turbulent to make the drop, acco

In [6]:
df["Sentences"] = df["Clean_content"].apply(sentences)
df

,Title,Link,Date,Clean_content,Sentences
0,'We're one cigarette away': Illegal weed farms...,https://www.nbcnews.com/news/us-news/fire-guns...,Thu Dec 02 2021 01:40:21 GMT+0000 (UTC),"LAKE ELSINORE, Calif. After a 21⁄2 mile trek t...","[LAKE ELSINORE, Calif. After a 21⁄2 mile trek ..."
1,Wildfire near Rocky Mountain National Park ful...,https://www.nbcnews.com/news/us-news/wildfire-...,Sun Nov 21 2021 03:09:39 GMT+0000 (UTC),A wildfire burning for four days in steep terr...,[A wildfire burning for four days in steep ter...
2,Thousands of giant sequoias killed in Californ...,https://www.nbcnews.com/science/science-news/t...,Fri Nov 19 2021 20:00:52 GMT+0000 (UTC),LOS ANGELES Lightning sparked wildfires killed...,[LOS ANGELES Lightning sparked wildfires kille...
3,Former California professor indicted on arson ...,https://www.nbcnews.com/news/us-news/former-ca...,Fri Nov 19 2021 11:57:32 GMT+0000 (UTC),A former California professor accused of setti...,[A former California professor accused of sett...
4,Colorado pilot fighting wildfire was making fi...,https://www.nbcnews.com/news/us-news/colorado-...,Thu Nov 18 2021 03:52:01 GMT+0000 (UTC),The pilot of a firefighting aircraft helping t...,[The pilot of a firefighting aircraft helping ...
...,...,...,...,...,...
108,Oregon prepares for 'mass fatality incident' a...,https://www.nbcnews.com/news/us-news/largest-w...,Sat Sep 12 2020 15:24:00 GMT+0000 (UTC),Officials in Oregon are preparing for a mass f...,[Officials in Oregon are preparing for a mass ...
109,New California law to make it easier for forme...,https://www.nbcnews.com/news/us-news/proposed-...,Fri Sep 11 2020 20:35:00 GMT+0000 (UTC),"OAKLAND, Calif.—Gov. Gavin Newsom signed a new...","[OAKLAND, Calif.—Gov., Gavin Newsom signed a n..."
110,Power outages hamper evacuation warnings and d...,https://www.nbcnews.com/business/business-news...,Fri Sep 11 2020 17:37:16 GMT+0000 (UTC),Danielle Mays lives in a wood cabin at the end...,[Danielle Mays lives in a wood cabin at the en...
111,7 deaths confirmed as wildfire rages in Northe...,https://www.nbcnews.com/news/us-news/7-bodies-...,Fri Sep 11 2020 05:30:00 GMT+0000 (UTC),Seven more deaths have been confirmed where a ...,[Seven more deaths have been confirmed where a...


In [7]:
nlp = spacy.load('en_core_web_sm') #Loading english NLP model
locations = {"Location":[]}
for sentence_list in df["Sentences"]:
    list_of_location = []
    for sentence in sentence_list:
        doc = nlp(sentence)
        for ent in doc.ents:
            if ent.label_ == 'GPE':
                if ent.text != "turkey": #Sad if statement to write but necessary as the nlp model doesn't tell the bird and the country appart.
                    if ent.text not in list_of_location:
                        list_of_location.append(ent.text)
    locations["Location"].append(list_of_location)


df['Location'] = pd.DataFrame(locations)
df

,Title,Link,Date,Clean_content,Sentences,Location
0,'We're one cigarette away': Illegal weed farms...,https://www.nbcnews.com/news/us-news/fire-guns...,Thu Dec 02 2021 01:40:21 GMT+0000 (UTC),"LAKE ELSINORE, Calif. After a 21⁄2 mile trek t...","[LAKE ELSINORE, Calif. After a 21⁄2 mile trek ...","[Calif., America, Los Angeles, California, U.S..."
1,Wildfire near Rocky Mountain National Park ful...,https://www.nbcnews.com/news/us-news/wildfire-...,Sun Nov 21 2021 03:09:39 GMT+0000 (UTC),A wildfire burning for four days in steep terr...,[A wildfire burning for four days in steep ter...,"[Colorado, The Kruger Rock Fire, Estes Park, D..."
2,Thousands of giant sequoias killed in Californ...,https://www.nbcnews.com/science/science-news/t...,Fri Nov 19 2021 20:00:52 GMT+0000 (UTC),LOS ANGELES Lightning sparked wildfires killed...,[LOS ANGELES Lightning sparked wildfires kille...,[California]
3,Former California professor indicted on arson ...,https://www.nbcnews.com/news/us-news/former-ca...,Fri Nov 19 2021 11:57:32 GMT+0000 (UTC),A former California professor accused of setti...,[A former California professor accused of sett...,"[California, San Jose, U.S.]"
4,Colorado pilot fighting wildfire was making fi...,https://www.nbcnews.com/news/us-news/colorado-...,Thu Nov 18 2021 03:52:01 GMT+0000 (UTC),The pilot of a firefighting aircraft helping t...,[The pilot of a firefighting aircraft helping ...,"[Colorado, the Kruger Rock Fire, Estes Park, L..."
...,...,...,...,...,...,...
108,Oregon prepares for 'mass fatality incident' a...,https://www.nbcnews.com/news/us-news/largest-w...,Sat Sep 12 2020 15:24:00 GMT+0000 (UTC),Officials in Oregon are preparing for a mass f...,[Officials in Oregon are preparing for a mass ...,"[Oregon, Salem, Washington, California, Sacram..."
109,New California law to make it easier for forme...,https://www.nbcnews.com/news/us-news/proposed-...,Fri Sep 11 2020 20:35:00 GMT+0000 (UTC),"OAKLAND, Calif.—Gov. Gavin Newsom signed a new...","[OAKLAND, Calif.—Gov., Gavin Newsom signed a n...","[OAKLAND, California, crimes,”https://t.co, Sa..."
110,Power outages hamper evacuation warnings and d...,https://www.nbcnews.com/business/business-news...,Fri Sep 11 2020 17:37:16 GMT+0000 (UTC),Danielle Mays lives in a wood cabin at the end...,[Danielle Mays lives in a wood cabin at the en...,"[California, North Complex West Zone, Butte Co..."
111,7 deaths confirmed as wildfire rages in Northe...,https://www.nbcnews.com/news/us-news/7-bodies-...,Fri Sep 11 2020 05:30:00 GMT+0000 (UTC),Seven more deaths have been confirmed where a ...,[Seven more deaths have been confirmed where a...,"[Butte County, California, Oregon, Phoenix, Me..."


In [8]:
locations = np.concatenate(df["Location"])

print(f"There are {len(locations)} locations")

d = {}

locator = geopy.geocoders.Nominatim(user_agent='mygeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
lattitudes = []
longitudes = []
for loc in locations :
    try :
        if loc not in d.keys():
            code = geocode(loc)
            lattitudes.append(code[1][0])
            longitudes.append(code[1][1])
            d[loc] = (code[1][0],code[1][1])
        else:
            coord = d[loc]
            lattitudes.append(coord[0])
            longitudes.append(coord[1])
    except :
        continue

folium_map = folium.Map(location=[59.338315,18.089960],
                        zoom_start=2,
                        tiles='CartoDB dark_matter')
FastMarkerCluster(data=list(zip(lattitudes, longitudes))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

There are 810 locations
